# Part 5: Running batch jobs with the exec interface

GRASS modules and user scripts can be executed in a GRASS GIS [non-interactive session](https://grass.osgeo.org/grass-stable/manuals/grass7.html) with `--exec`.

Here is a simple call to list all available rasters in PERMANENT mapset:

In [ ]:
%%bash
grass ~/grassdata/nc_spm_08_grass7/PERMANENT --exec g.list type=vector mapset=PERMANENT -t

We can also provide a script. Here we create a simple Python script computing viewshed which requires 3 parameters (id, x and y coordinate) and write it to a file `myscript.py`. The desired output is a PNG rendering of the viewshed:

In [ ]:
%%writefile myscript.py
import sys
import grass.script as gs
import grass.jupyter as gj


def main(cat, x, y):
    gs.run_command("g.region", raster="elevation")
    gs.run_command("r.viewshed", input="elevation", output=f"viewshed_{cat}", coordinates=(x, y),
                   observer_elevation=3, quiet=True, overwrite=True)
    img = gj.GrassRenderer(filename=f"viewshed_{cat}.png")
    img.d_rast(map="elevation")
    img.d_rast(map=f"viewshed_{cat}")

if __name__ == "__main__":
    args = sys.argv[1:]
    main(*args)


Then, we call it with different parameters. In this case, since we are only interested in the output PNG file, we can use a temporary mapset. In this way, the runs won't interfere and can be parallelized, for example using [GNU Parallel](https://www.gnu.org/software/parallel/) (e.g., `parallel --jobs 10 < jobs.txt`).

In [ ]:
%%bash

grass --tmp-mapset ~/grassdata/nc_spm_08_grass7 --exec python3 myscript.py 1 633298 224283
grass --tmp-mapset ~/grassdata/nc_spm_08_grass7 --exec python3 myscript.py 2 639805 224410
grass --tmp-mapset ~/grassdata/nc_spm_08_grass7 --exec python3 myscript.py 3 632788 217958
grass --tmp-mapset ~/grassdata/nc_spm_08_grass7 --exec python3 myscript.py 4 640844 218852

Check one of the resulting PNG files:

In [ ]:
from IPython.display import Image

Image("viewshed_2.png")